In [1]:
import csv
import cv2
import numpy as np

lines = []
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

In [2]:
images = []
measurements = []

# Correction factor
correction = 0.2

for line in lines:
    steering_center = float(line[3])

    for i in range(3):
        source_path = line[i]
        filename = source_path.split('/')[-1]
        current_path = 'data/IMG/' + filename
#     print(current_path)
        image = cv2.imread(current_path)
        images.append(image)
#     print(measurement)
        if i == 0:
            measurements.append(steering_center)
        elif i == 1:
            measurements.append(steering_center + correction)  # streering_left
        elif i == 2:
            measurements.append(steering_center - correction) # steering_right
    

In [3]:
# Data Augmentation
augmented_images, augmented_measurements = [], []
for image, measurement in zip(images, measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    # Flip the images
    augmented_images.append(cv2.flip(image, 1))
    # change the direction of the measurement
    augmented_measurements.append(measurement*-1.0)

In [4]:


X_train = np.array(augmented_images)
y_train = np.array(augmented_measurements)


In [5]:
X_train.shape

(14016, 160, 320, 3)

In [6]:
y_train.shape

(14016,)

In [9]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers import Conv2D, MaxPool2D
from keras.layers import Cropping2D

In [13]:
model = Sequential()
# normalize the input data
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160, 320, 3)))

# Cropping the top 70 top pixels and 25 bottom pixels.
model.add(Cropping2D(cropping=((70, 25), (0, 0))))

model.add(Conv2D(24, (5, 5), strides=(2,2), activation='relu'))
model.add(Conv2D(36, (5, 5), strides=(2,2), activation='relu'))
model.add(Conv2D(48, (5, 5), strides=(2,2), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=5)

model.save('model.h5')

/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="relu", strides=(2, 2))`
  
/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="relu", strides=(2, 2))`
  if __name__ == '__main__':
/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="relu", strides=(2, 2))`
  # Remove the CWD from sys.path while we load stuff.


Train on 11212 samples, validate on 2804 samples
Epoch 1/5
11212/11212 [==============================] - 124s - loss: 0.0253 - val_loss: 0.0521
Epoch 2/5
11212/11212 [==============================] - 122s - loss: 0.0184 - val_loss: 0.0487
Epoch 3/5
11212/11212 [==============================] - 122s - loss: 0.0156 - val_loss: 0.0489
Epoch 4/5
11212/11212 [==============================] - 124s - loss: 0.0134 - val_loss: 0.0490
Epoch 5/5
11212/11212 [==============================] - 119s - loss: 0.0119 - val_loss: 0.0446
